In [1]:
import numpy as np
import cv2
import os
from PIL import Image, ImageDraw, ImageFont

class SpeedRead:

    def __init__(self, document_path, video_path=None,
                 fps=5, edge=1.2, countdown = 0,
                 font_name='times', font_size=36,
                 color=(0,0,0), background_color=(255,255,255)):
        '''
        Inputs:
        - document_path (str) - valid path to text document
        - video_path (str) - (Optional) path to save video (default same name as document_path)
        - fps (int) - (Optional) number of words per second (default 5)
        - countdown (int) - (Optional) number of seconds to countdown (default 0)
        - font_name (str) - (Optional) name of the font to use (default 'times')
        - font_size (int) - (Optional) size of the font to use (default 36)
        - edge (float) - (Optional) relative size of edging (default 1.2), edge >= 1
        - color (tuple or list) - (Optional) color of the text to use in RGB mode (default (0,0,0) - black)
        '''
        self.document_path = self.set_document_path(document_path)
        self.video_path = self.set_video_path(video_path)
        self.font = self.set_font(font_name,font_size)
        self.fps = self.set_number(fps)
        self.countdown = self.set_number(countdown)
        self.edge = self.set_edge(edge)
        self.color = self.set_color(color)
        self.background_color = self.set_color(background_color)
        self.max_len = self.set_max_length()
        self.max_width = self.set_max_width()
        self.size = self.set_image_size()
        print('To create a video call create_video method')
        
    def set_document_path(self,document_path):
        _, file_extension = os.path.splitext(document_path)
        if os.path.isfile(document_path):
            if file_extension == '.txt':
                return document_path
            else:
                raise TypeError('Document should be of .txt type')
        else:
            raise FileNotFoundError('Desired document does not exist')
    
    def set_video_path(self,video_path):
        if video_path:
            if video_path.endswith('.mp4') or video_path.endswith('.avi'):
                os.makedirs(video_path, exist_ok = True)
                return video_path
            else:
                videoname, _ = os.path.splitext(video_path)
                video_path = videoname + '.mp4'
                os.makedirs(video_path, exist_ok = True)
                print('Desired video format is not supported: reset to mp4')
                print('Path to the video is set to:',video_path)
                return video_path
        filename, _ = os.path.splitext(self.document_path)
        video_path = filename + '.avi'
        print('Default path to the video is set to:',video_path)
        return video_path

    def set_font(self,font_name,font_size):
        try:
            font = ImageFont.truetype(font_name + '.ttf', font_size)
        except:
            font = ImageFont.truetype('times.ttf', font_size)
            print("Desired font not found: font reset to default ('times')")
        return font
    
    def set_edge(self,edge):
        if float(edge) >= 1:
            return float(edge)
        raise ValueError('Edge should be >= 1')
    
    def set_color(self,color):
        if len(color) == 3 and sum([(c >= 0) and (c <= 255) and (type(c) == int) for c in color]) == 3:
            return color
        else:
            raise TypeError('Color should be in RGB mode')

    def set_max_length(self):
        return max([self.font.getbbox(word)[2] for word in self.get_word_list()])

    def set_max_width(self):
        return max([self.font.getbbox(word)[3] for word in self.get_word_list()])

    def set_image_size(self):
        return ([int(self.max_len*self.edge),int(self.max_width*self.edge)])

    def set_number(self,fps):
        if type(fps) == int:
            if fps > 0:
                return fps
        raise TypeError('number of words per second (fps) should be integer')

    def get_data(self):
        with open(self.document_path,'r') as f:
            data = ' '.join(line.rstrip() for line in f.readlines())
        return data
    
    def get_word_list(self):
        return self.get_data().split()
    
    def add_text_to_image(self,word,color,background_color):
        image = np.ones([self.size[1],self.size[0],3],dtype=np.uint8)*np.array(background_color,dtype=np.uint8)
        left = int((image.shape[1] - self.font.getbbox(word)[2]) / 2)
        bottom = int((image.shape[0] - self.max_width) / 2)
        image = Image.fromarray(image)
        draw = ImageDraw.Draw(image)
        draw.text((left, bottom), word, color=color, fill=color, font=self.font)
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    
    def create_video(self, video_path=None, fps=None, color=None, background_color=None, countdown=None):
        '''
        Inputs:
        - video_path (str) - (Optional) path to save video (default video_path was set during initialization)
        - fps (int) - (Optional) number of words per second (default fps was set during initialization)
        '''

        video_path = self.set_video_path(video_path) if video_path else self.video_path
        fps = self.set_fps(fps) if fps else self.fps
        color = self.set_color(color) if color else self.color
        background_color = self.set_color(background_color) if background_color else self.background_color
        countdown = self.set_number(countdown) if countdown else self.countdown

        out = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, self.size)

        for i in range(countdown,0,-1):
            image = self.add_text_to_image(str(i),color,background_color)
            for _ in range(fps):
                out.write(image)

        for word in self.get_word_list():
            image = self.add_text_to_image(word,color,background_color)
            out.write(image)

        out.release()
        print(f'Video was saved to {video_path}')

In [ ]:
sr = SpeedRead('/path_to_my_txt_file/filename.txt',font_size=100,countdown=3,fps=5,background_color=(0,0,0),color=(255,0,0))

In [ ]:
sr.create_video()